# Stable Diffusion Videos

This notebook allows you to generate videos by interpolating the latent space of [Stable Diffusion](https://github.com/CompVis/stable-diffusion).

You can either dream up different versions of the same prompt, or morph between different text prompts (with seeds set for each for reproducibility).

If you like this notebook:
- consider giving the [repo a star](https://github.com/nateraw/stable-diffusion-videos) ⭐️
- consider following me on Github [@nateraw](https://github.com/nateraw) 

You can file any issues/feature requests [here](https://github.com/nateraw/stable-diffusion-videos/issues)

Enjoy 🤗

## Setup

In [1]:
%%capture
! pip install stable_diffusion_videos[realesrgan]
! git config --global credential.helper store

## Authenticate with Hugging Face Hub

You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work. For more information on access tokens, please refer to [this section of the documentation](https://huggingface.co/docs/hub/security-tokens).

  > ⚠️ **Important**: You must also go to the [model repository](https://huggingface.co/CompVis/stable-diffusion-v1-4) and click "Access Repository" so you can download the model.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


## Run the App 🚀

### Load the Interface

This step will take a couple minutes the first time you run it.

In [ ]:
import torch

from stable_diffusion_videos import StableDiffusionWalkPipeline, Interface

pipeline = StableDiffusionWalkPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16,
    revision="fp16",
).to("cuda")

interface = Interface(pipeline)

In [ ]:
#@title Connect to Google Drive to Save Outputs

#@markdown If you want to connect Google Drive, click the checkbox below and run this cell. You'll be prompted to authenticate.

#@markdown If you just want to save your outputs in this Colab session, don't worry about this cell

connect_google_drive = True #@param {type:"boolean"}

#@markdown Then, in the interface, use this path as the `output` in the Video tab to save your videos to Google Drive:

#@markdown > /content/gdrive/MyDrive/stable_diffusion_videos


if connect_google_drive:
    from google.colab import drive

    drive.mount('/content/gdrive')

In [5]:
#SaveSave file to google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

---

## Use `walk` programmatically

The other option is to not use the interface, and instead use `walk` programmatically. Here's how you would do that...

First we define a helper fn for visualizing videos in colab

In [6]:
from IPython.display import HTML
from base64 import b64encode

def visualize_video_colab(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

In [7]:
# load prompts
import numpy as np
prompts =np.load('prompts.npy', 'r')
prompts.files
prompts_b = prompts['prompts_b']
prompts_b.shape, prompts_b[0]

((101,), ' "A boy", "is running", "on the lawn."')

Walk! 🚶‍♀️

In [8]:
import os
from google.colab import files

# generate videos from prompts and save them
def v2tDiffusion(prompt):
    # generate videos
    video_path = pipeline.walk(
        prompt,
        [42, 1337],
        fps=8,                      # use 5 for testing, 25 or 30 for better quality
        num_interpolation_steps=32,  # use 3-5 for testing, 30 or more for better results
        height=480,                 # use multiples of 64 if > 512. Multiples of 8 if < 512.
        width=480,                  # use multiples of 64 if > 512. Multiples of 8 if < 512.
    )

    return video_path

In [ ]:
# batch generate and save videos
for i, prompt in enumerate(prompts_b):
    # reform the prompt
    raw = prompts_b[i].split(",")
    prompts_re = []
    for s in raw:
        prompts_re.append(s[2:-1])

    # new name for videos
    fname = f'{i}.mp4'

    # generate videos
    video_path = v2tDiffusion(prompts_re)

    # save videos to gdrive
    os.rename(video_path, fname)
    !cp -r "/content/{fname}" "/content/gdrive/MyDrive/diffvideos"
    # files.download(fname)  ## to local
    
    if i % 10 == 0:
        print(f"Finish generating {i+1} videos \n")